# PyTorch Quickstart

## Setup for run
This Notebook is a test run of PyTorch tutorial targeted for running on AMD GPU.<br>
Assuming PyTorch for AMD ROCm is available, for example, inside a appropriately configured Docker container.
For example, a Docker image with a tag `rocm/pytorch:rocm5.4_ubuntu20.04_py3.8_pytorch_1.12.1` is used.
And the container is created with this command:
```
docker run -it --device=/dev/kfd --device=/dev/dri --security-opt seccomp=unconfined \
  --group-add video -p 11088:8988 -v $HOME:/myhome --name my-pt-test \
  rocm/pytorch:rocm5.4_ubuntu20.04_py3.8_pytorch_1.12.1 /bin/bash
```

## Working with data

In [9]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Download training and test data from open datasets.

In [10]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

Create DataLoader by wrapping Dataset, which facilitate handling like batching, sampling, shuffling and multiprocess data loading.

In [11]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W]: {X.shape}')
    print(f'Shape of y: {y.shape} {y.dtype}')
    break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

Create class inheriting `nn.Module`

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [30]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [31]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing model parameters

In [32]:
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Single training loop

In [33]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    model.train()  # set train mode
    
    for batch, (X, y) in enumerate(dataloader):
        
        X, y = X.to(device), y.to(device)
        
        # compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # back-prop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [34]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    model.eval()   # set eval(test) mode
    
    test_loss, correct = 0, 0
    
    with torch.no_grad():   # no gradient calculation during test
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')
        

In [35]:
epochs = 50

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print('Done!')

Epoch 1
-------------------------------
loss: 2.308357 [    0/60000]
loss: 2.298764 [ 6400/60000]
loss: 2.281631 [12800/60000]
loss: 2.271461 [19200/60000]
loss: 2.253024 [25600/60000]
loss: 2.217248 [32000/60000]
loss: 2.227242 [38400/60000]
loss: 2.191671 [44800/60000]
loss: 2.188478 [51200/60000]
loss: 2.146126 [57600/60000]
Test Error: 
 Accuracy: 39.0%, Avg loss: 2.155049 

Epoch 2
-------------------------------
loss: 2.167293 [    0/60000]
loss: 2.160949 [ 6400/60000]
loss: 2.101174 [12800/60000]
loss: 2.113639 [19200/60000]
loss: 2.070256 [25600/60000]
loss: 1.998778 [32000/60000]
loss: 2.022470 [38400/60000]
loss: 1.946807 [44800/60000]
loss: 1.949832 [51200/60000]
loss: 1.867178 [57600/60000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 1.878420 

Epoch 3
-------------------------------
loss: 1.914675 [    0/60000]
loss: 1.885818 [ 6400/60000]
loss: 1.765454 [12800/60000]
loss: 1.803469 [19200/60000]
loss: 1.704124 [25600/60000]
loss: 1.645429 [32000/60000]
loss: 1.659986 [38400/

## Saving Models

In [36]:
torch.save(model.state_dict(), 'model.pth')
print('Saved PyTorch model state to model.pth')

Saved PyTorch model state to model.pth


## Loading Models

In [37]:
model = NeuralNetwork()

model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

Making predictions with the loaded model

In [41]:
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot',]

model.eval()   # set inference mode

x, y = test_data[201][0], test_data[201][1]

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "T-shirt/top", Actual: "T-shirt/top"
